In [1]:
from bs4 import BeautifulSoup
import requests
import re
import datetime

In [2]:
match_url = "https://www.cricbuzz.com/cricket-match-squads/77863/rsa-vs-ind-2nd-t20i-india-tour-of-south-africa-2023-24"
# started_match_url = "https://www.cricbuzz.com/cricket-match-squads/77863/rsa-vs-ind-2nd-t20i-india-tour-of-south-africa-2023-24"

In [3]:
COUNTRY_MAP = {
    "afghanistan": "AFG",
    "australia": "AUS",
    "bangladesh": "BAN",
    "england": "ENG",
    "india": "IND",
    "ireland": "IRE",
    "new zealand": "NZ",
    "pakistan": "PAK",
    "south africa": "SA",
    "sri lanka": "SL",
    "west indies": "WI",
    "zimbabwe": "ZIM",
    "netherlands": "NED"
}

In [4]:
website_html = requests.get(match_url)
soup = BeautifulSoup(website_html.content, 'html.parser')

In [5]:
# Get the match title

p_match_title = re.compile(r'\bcb-nav-hdr\b')
match_full_title = soup.find('h1', class_=p_match_title).text
match_title = match_full_title.split(',')[0]
team1 = match_title.split('vs')[0].strip()
team2 = match_title.split('vs')[1].strip()
team1_code = COUNTRY_MAP[team1.lower()]
team2_code = COUNTRY_MAP[team2.lower()]

In [6]:
p_series = re.compile(r'\bcb-nav-subhdr\b')
match_info_soup = soup.find('div', class_=p_series)
series = match_info_soup.find_all('a')[0]['title']
venue = match_info_soup.find_all('a')[1]['title']
match_start_date_time = match_info_soup.find('span', {'itemprop': 'startDate'})['content']
start_date = datetime.datetime.strptime(match_start_date_time, '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d')
start_time = datetime.datetime.strptime(match_start_date_time, '%Y-%m-%dT%H:%M:%S%z').strftime('%H:%M:%S') # time is in UTC
end_date = match_info_soup.find('span', {'itemprop': 'endDate'})['content']

In [7]:
# Playing 11 

cricbuzz_match_id = match_url.split('/')[-2]
squad_url = f"https://www.cricbuzz.com/api/html/match-squads/{cricbuzz_match_id}"
squad_html = requests.get(squad_url)
squad_soup = BeautifulSoup(squad_html.content, 'html.parser')

In [ ]:
# team1_sqaud = {
#     "name": ""
#     "players": {
#         "player_name": "player_role",
#         .....
#     }
# }

# same for team2

In [8]:
def get_playing11(playing11_list):
    team_sqaud = {
        'playing11': {}
    }

    for player in playing11_list:
        player_role = player.find_next('div').find_next('div').find_next('div').find('span', {'class':'cb-font-12 text-gray'}).text.strip()
        player_name_role = player.find_next('div').find_next('div').find_next('div').text.strip()
        player_name = player_name_role.replace(player_role, '').strip()
        if '(C)' in player_name: 
            player_name = player_name.replace('(C)', '').strip()
            team_sqaud['captain'] = player_name
        elif '(WK)' in player_name:
            player_name = player_name.replace('(WK)', '').strip()
            team_sqaud['wicket_keeper'] = player_name
        elif '(C & WK)' in player_name:
            player_name = player_name.replace('(C & WK)', '').strip()
            team_sqaud['captain'] = player_name
            team_sqaud['wicket_keeper'] = player_name
        team_sqaud['playing11'][player_name] = player_role
        
    return team_sqaud

In [13]:
def get_bench_players(bench_players_list):
    team_squad = {
        'bench_players': {}
    }

    for player in bench_players_list:
        player_role = player.find_next('div').find_next('div').find_next('div').find('span', {
            'class': 'cb-font-12 text-gray'}).text.strip()
        player_name_role = player.find_next('div').find_next('div').find_next('div').text.strip()
        player_name = player_name_role.replace(player_role, '').strip()
        if '(WK)' in player_name:
            player_name = player_name.replace('(WK)', '').strip()
        team_squad['bench_players'][player_name] = player_role

    return team_squad

In [14]:
team1_sqaud = {
    'playing11': {}
}
team2_sqaud = {
    'playing11': {}
}

team1_sqaud['country_code'] = squad_soup.find('a', {'class': 'cb-team1'}).find_next('div').find_next('div').text.strip()
team2_sqaud['country_code'] = squad_soup.find('a', {'class': 'cb-team2'}).find_next('div').find_next('div').text.strip()

team1_playing11_list_soup = squad_soup.find('div', {'class': 'cb-col cb-col-50 cb-play11-lft-col'}).find_all('a')
team2_playing11_list_soup = squad_soup.find('div', {'class': 'cb-col cb-col-50 cb-play11-rt-col'}).find_all('a')

team1_playing11_sqaud = get_playing11(team1_playing11_list_soup)
team2_playing11_sqaud = get_playing11(team2_playing11_list_soup)

team1_bench_players_list_soup = squad_soup.find_all('div', {'class': 'cb-col cb-col-50 cb-play11-lft-col'})[
            1].find_all('a')
team2_bench_players_list_soup = squad_soup.find_all('div', {'class': 'cb-col cb-col-50 cb-play11-rt-col'})[
            1].find_all('a')

team1_bench_players_squad = get_bench_players(team1_bench_players_list_soup)
team2_bench_players_squad = get_bench_players(team2_bench_players_list_soup)

In [16]:
team2_bench_players_squad

{'bench_players': {'Shreyas Iyer': 'Batter',
  'Deepak Chahar': 'Bowler',
  'Ravi Bishnoi': 'Bowler',
  'Ishan Kishan': 'WK-Batter',
  'Washington Sundar': 'Batting Allrounder',
  'Ruturaj Gaikwad': 'Batter'}}

In [17]:
len(squad_soup.find_all('div', {'class': 'cb-col cb-col-50 cb-play11-lft-col'})) == 3

False